In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
import time

In [22]:
import os
import time
import shutil
import random
from datetime import date

In [ ]:
def timing_val(func):
    def wrapper(*arg, **kw):
        '''source: http://www.daniweb.com/code/snippet368.html'''
        t1 = time.time()
        func(*arg, **kw)
        t2 = time.time()
        segs = int(t2 - t1)
        return f"tardó {segs} segs..."
    return wrapper


In [23]:
def delete_files_from_dir(dir:str):
    print(f"Borrando el directorio: {dir}")
    for root, dirs, files in os.walk(dir):
        for f in files:
            os.unlink(os.path.join(root, f))
        for d in dirs:
            shutil.rmtree(os.path.join(root, d))

In [ ]:
def download_wait(directory:str, timeout:int, prefix:str, nfiles:int=1):
    """
    Wait for downloads to finish with a specified timeout.

    Args
    ----
    directory : str
        The path to the folder where the files will be downloaded.
    timeout : int
        How many seconds to wait until timing out.
    nfiles : int, defaults to None
        If provided, also wait for the expected number of files.

    """

    def rename_files(prefix:str, dir:str):
        for filename in os.listdir(dir):
            new_filename = f"{prefix}_{filename}"
            new_filename = new_filename.replace(" ","_").strip()
            os.rename(dir+"/"+filename, dir+"/"+new_filename)

    print("Esperando descarga")

    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        if len(files) < nfiles:
            dl_wait = True

        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True

        seconds += 1

    if seconds >= timeout:
        print("Error esperando descarga, se superó el tiempo de espera.")
        delete_files_from_dir("./temp")
    else:
        print("Archivo descargado")
        if seconds < 15:
            time.sleep( random.randint(3, 6) )

        print("Renombrando archivo")
        rename_files(prefix, directory)
        time.sleep(5)
        print(f"Subiendo Archivos")
        #Subir_File(AQUI DEBE IR LA FUNCIÓN QUE SUBE LOS ARCHIVOS)

        delete_files_from_dir("./temp")


In [ ]:

download_wait(directory="./temp", timeout=3)

In [24]:
delete_files_from_dir("./temp")

Borrando el directorio: ./temp


In [ ]:
os.listdir("./temp")

In [ ]:
options = webdriver.ChromeOptions()
#options.add_argument('headless')
#options.add_argument("--headless")
prefs = {'download.default_directory' : './temp/anexos/'}
options.add_experimental_option('prefs', prefs)
options.add_argument("--window-size=1024,768")
options.add_argument("--no-sandbox")
options.add_argument("start-maximized")
options.add_argument("--disable-infobars")

In [ ]:
driver = webdriver.Chrome(options=options,
            service=Service(executable_path=ChromeDriverManager().install()),
        )

In [ ]:
driver.get("https://upcp-compranet.hacienda.gob.mx/sitiopublico/#/")

In [ ]:
driver.find_element(By.XPATH,'//*[@name="proceso"]').click()

In [ ]:
driver.find_element(By.XPATH,'//*[text()="PROCEDIMIENTO DE CONTRATACIÓN"]').click()

In [ ]:
driver.find_element(By.XPATH,'//*[@name="ley"]').click()

In [ ]:
for i in range(0, 10):
    next_page_btn = driver.find_element(By.XPATH,'//button[contains(@class,"p-paginator-next")]')
    if "p-disabled" in next_page_btn.get_attribute("class"):
        print("Se acabaron las páginas disponibles")
        break
    else:
        ActionChains(driver).scroll_to_element(next_page_btn).pause(1).click().perform()




In [ ]:
driver.find_element(By.XPATH,'//*[@name="contratacion"]').click()

In [ ]:
driver.find_element(By.XPATH,'//*[(text()="ADQUISICIONES")]').click()

In [ ]:
click = lambda xpath: driver.find_element(By.XPATH, xpath).click()

In [ ]:
click('//*[@label="Filtros"]/button')

In [ ]:
click('//*[@name="claves"]')

In [ ]:
claves = ["25401","25301","25501","32401","53101","53201"]

In [ ]:
for clave in claves:
    driver.find_element(By.XPATH,'//p-multiselect[@name="claves"]//input[contains(@class,"filter")]').send_keys(clave)
    time.sleep(2)
    
    click(f'//*[contains(text(),"{clave}")]')
    time.sleep(2)
    
    driver.find_element(By.XPATH,'//p-multiselect[@name="claves"]//input[contains(@class,"filter")]').clear()
    time.sleep(1)
    

In [ ]:
click('//span[text()="Buscar"]')

In [ ]:
def esperar_carga_componenete(driver):
    try:
        WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="spinner"]')))
    except TimeoutException:
        print("OJO, no apareció la leyenda de carga")
    try:
        WebDriverWait(driver, 120).until(EC.invisibility_of_element_located((By.XPATH, '//div[@class="spinner"]')))
    except TimeoutException:
        print("Error: No desaparece leyenda cargando")
    
    time.sleep(1)

In [ ]:
def duerme(a:int, b:int=0):
    "Sleeps for @a seconds, if b is given it'll sleep for a random between @a, @b"
    time_to_sleep = random.randint(a, b) if b else a
    time.sleep(time_to_sleep)

In [ ]:
for i in range(0, 10):
    next_page_btn = driver.find_element(By.XPATH,'//button[contains(@class,"p-paginator-next")]')
    if "p-disabled" in next_page_btn.get_attribute("class"):
        print("Se acabaron las páginas disponibles")
        break
    else:
        ActionChains(driver).scroll_to_element(next_page_btn)\
            .pause(1).click(next_page_btn).perform()
        esperar_carga_componenete(driver)
        duerme(2, 4)

In [6]:
import pandas as pd

In [10]:
df = pd.read_csv("concluidos.csv", usecols = ['num_proc'])

In [19]:
df[df['num_proc'] == "AA-50-GYR-050GYR016-I-79-2023"] 

,num_proc
2,AA-50-GYR-050GYR016-I-79-2023


In [17]:
df['num_proc'].str.contains("AA-50-GYR-050GYR016-I-79-2023").any()

np.True_

In [21]:
if df['num_proc'].str.contains("AA-50-GYR-050GYR016-I-79-2023").any():
    print("simon")

In [ ]:
rows = driver.find_elements(By.XPATH, '//td[@class="p-link2"]')
ops_found = len(rows)
print(f"se encontraron {ops_found} oportunidades en está página")

In [ ]:
rows = driver.find_elements(By.XPATH, '//td[@class="p-link2"]')
ops_found = len(rows)
print(f"se encontraron {ops_found} oportunidades en está página")
for i in range(0, ops_found):
    rows = driver.find_elements(By.XPATH, '//td[@class="p-link2"]')
    rows[i].click()
    esperar_carga_componenete(driver)

    print(f"Extrayendo inf de la oportunidad {i+1}")
    #Código del expediente
    cod_exp = driver.find_element(By.XPATH, '//label[text()="Código del expediente:"]/following-sibling::label').text
    #Número de procedimiento de contratación
    proc_cont = driver.find_element(By.XPATH, '//label[text()="Número de procedimiento de contratación:"]/following-sibling::label').text
    #Dependencia (SOLO TEXTO DESPUÉS DEL GUIÓN “-”)
    dependencia = driver.find_element(By.XPATH, '//label[text()="Dependencia o Entidad:"]/following-sibling::label').text
    #Unidad Compradora (SOLO CÓDIGO IDENTIDICADOR)
    unidad_comp = driver.find_element(By.XPATH, '//label[text()="Unidad compradora"]/following-sibling::label').text
    #Correo electrónico Unidad compradora
    correo = driver.find_element(By.XPATH, '//label[text()="Correo electrónico unidad compradora:"]/following-sibling::label').text
    # Nombre del procedimiento de contratación
    nombre_proceidmiento = driver.find_element(By.XPATH, '//label[text()="Nombre del procedimiento de contratación:"]/following-sibling::label').text
    # Tipo de procedimiento
    tipo_proc = driver.find_element(By.XPATH, '//label[text()="Tipo de procedimiento de contratación:"]/following-sibling::label').text
    # Entidad Federativa
    entidad_fed = driver.find_element(By.XPATH, '//label[text()="Entidad Federativa donde se llevará a cabo la contratación:"]/following-sibling::label').text
    # Año del ejercicio presupuestal
    anio_ej = driver.find_element(By.XPATH, '//label[text()="Año del ejercicio presupuestal:"]/following-sibling::label').text
    # Procedimiento exclusivo para MiPymes
    proc_exc = driver.find_element(By.XPATH, '//label[text()="Procedimiento exclusivo para MIPYMES:"]/following-sibling::label').text
    # Fecha y hora de publicación:
    fecha_pub = driver.find_element(By.XPATH, '//label[text()="Fecha y hora de publicación:"]/following-sibling::label').text
    # Lista de Claves
    claves_list = driver.find_elements(By.XPATH, '//th[text()="Clave"]/ancestor::thead/following-sibling::tbody/tr/td[1]')
    claves = ",".join([i.text for i in claves])
    # Participación de Testigo Social
    part_testigo = driver.find_element(By.XPATH, '//label[text()="Participación de Testigo Social:"]/following-sibling::label').text
    # Abastecimiento simultaneo
    abastecimiento_sim = driver.find_element(By.XPATH, '//label[text()="Abastecimiento simultáneo:"]/following-sibling::label').text
    # Plurianual
    plurianual = driver.find_element(By.XPATH, '//label[text()="Es plurianual:"]/following-sibling::label').text
    # Tipo de contratación
    tipo_cont = driver.find_element(By.XPATH, '//label[text()="Tipo de contratación:"]/following-sibling::label').text
    # Anticipo
    anticipo = driver.find_element(By.XPATH, '//label[text()="Anticipo:"]/following-sibling::label').text
    # Forma de Pago
    forma_pago = driver.find_element(By.XPATH, '//label[text()="Forma de pago:"]/following-sibling::label').text
    # Plazo para entrega de garantía
    plazo_garant = driver.find_element(By.XPATH, '//label[text()="Plazo en días para entregar la garantía:"]/following-sibling::label').text
    # Número de meses para la garantía
    meses_garant = driver.find_element(By.XPATH, '//label[text()="Número de meses que debe cumplir la garantía:"]/following-sibling::label').text
    # Carácter
    caracter = driver.find_element(By.XPATH, '//label[text()="Carácter:"]/following-sibling::label').text
    # Caso fortuito o fuerza mayor
    caso_fort  = driver.find_element(By.XPATH, '//label[text()="Caso fortuito o fuerza mayor:"]/following-sibling::label').text
    # Tipo de contrato abierto
    tipo_cont_abierto = driver.find_element(By.XPATH, '//label[text()="Tipo de contrato abierto:"]/following-sibling::label').text
    
    break
    driver.back()
    esperar_carga_componenete(driver)


In [ ]:
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": r"C:\Users\juan-\Desktop\CNET Scrapping 2024\temp\\",
    "download.directory_upgrade": True,
    "download.prompt_for_download": False,
}
options.add_experimental_option("prefs", prefs)

options.add_argument("--window-size=1024,768")
options.add_argument("--no-sandbox")
options.add_argument("start-maximized")
options.add_argument("--disable-infobars")

driver = webdriver.Chrome(options=options,
            service=Service(executable_path=ChromeDriverManager().install()),
        )

In [ ]:
driver.get("https://upcp-compranet.hacienda.gob.mx/sitiopublico/#/sitiopublico/detalle/1e90d8d1aea64d2ca428c26c5506f4a3/procedimiento")

In [ ]:
click('//*[text()="Anuncios concluidos"]')

In [ ]:
prefix = cod_exp.rsplit("-", 1)[1]
for i in range(0, 10):
    anexos = driver.find_elements(By.XPATH,'//*[@ptooltip="Descargar archivo"]')
    for anexo in anexos:
        anexo.click()
        download_wait(directory="./temp", timeout=10, prefix=prefix)
    
    next_page_btn = driver.find_element(By.XPATH,'//*[@key="anexos"]/following-sibling::div//button[contains(@class,"p-paginator-next")]')
    if "p-disabled" in next_page_btn.get_attribute("class"):
        print("No hay más páginas de anexos disponibles")
        break
    else:
        ActionChains(driver).scroll_to_element(next_page_btn)\
            .pause(1).click(next_page_btn).perform()
        esperar_carga_componenete(driver)
        duerme(2, 4)

In [ ]:
dir = "./temp"
prefix = cod_exp.rsplit("-", 1)[1]

for filename in os.listdir(dir):
    new_filename = f"{prefix}_{filename}"
    new_filename = new_filename.replace(" ","_").strip()
    os.rename(dir+"/"+filename, dir+"/"+new_filename)

In [ ]:
def get_text_by_xpath(xp:str, required:bool=False, join:bool=False):
    try:
        if not join:
            txt = driver.find_element(By.XPATH, xp).text
        else:
            finds = driver.find_elements(By.XPATH, xp)
            txt = ",".join([i.text for i in finds])

        return txt
    
    except:
        if required:
            return False
        else:
            return ""
    

In [ ]:
cod_exp.rsplit("-", 1)[1]

In [ ]:
#Código del expediente
cod_exp = get_text_by_xpath(xp='//label[text()="Código del expediente:"]/following-sibling::label', required=True)
#Número de procedimiento de contratación
num_proc = get_text_by_xpath(xp='//label[text()="Número de procedimiento de contratación:"]/following-sibling::label', required=True)
#Dependencia (SOLO TEXTO DESPUÉS DEL GUIÓN “-”)
dependencia = get_text_by_xpath(xp='//label[text()="Dependencia o Entidad:"]/following-sibling::label')
#Unidad Compradora (SOLO CÓDIGO IDENTIDICADOR)
unidad_comp = get_text_by_xpath(xp='//label[text()="Unidad compradora"]/following-sibling::label')
#Correo electrónico Unidad compradora
correo = get_text_by_xpath(xp='//label[text()="Correo electrónico unidad compradora:"]/following-sibling::label')
# Nombre del procedimiento de contratación
nombre_procedimiento = get_text_by_xpath(xp='//label[text()="Nombre del procedimiento de contratación:"]/following-sibling::label')
# Tipo de procedimiento
tipo_proc = get_text_by_xpath(xp='//label[text()="Tipo de procedimiento de contratación:"]/following-sibling::label')
# Entidad Federativa
entidad_fed = get_text_by_xpath(xp='//label[text()="Entidad Federativa donde se llevará a cabo la contratación:"]/following-sibling::label')
# Año del ejercicio presupuestal
anio_ej = get_text_by_xpath(xp='//label[text()="Año del ejercicio presupuestal:"]/following-sibling::label')
# Procedimiento exclusivo para MiPymes
proc_exc = get_text_by_xpath(xp='//label[text()="Procedimiento exclusivo para MIPYMES:"]/following-sibling::label')
# Fecha y hora de publicación:
fecha_pub = get_text_by_xpath(xp='//label[text()="Fecha y hora de publicación:"]/following-sibling::label')
# Lista de Claves
claves_list = get_text_by_xpath(xp='//th[text()="Clave"]/ancestor::thead/following-sibling::tbody/tr/td[1]')
# Participación de Testigo Social
part_testigo = get_text_by_xpath(xp='//label[text()="Participación de Testigo Social:"]/following-sibling::label')
# Abastecimiento simultaneo
abastecimiento_sim = get_text_by_xpath(xp='//label[text()="Abastecimiento simultáneo:"]/following-sibling::label')
# Plurianual
plurianual = get_text_by_xpath(xp='//label[text()="Es plurianual:"]/following-sibling::label')
# Tipo de contratación
tipo_cont = get_text_by_xpath(xp='//label[text()="Tipo de contratación:"]/following-sibling::label')
# Anticipo
anticipo = get_text_by_xpath(xp='//label[text()="Anticipo:"]/following-sibling::label')
# Forma de Pago
forma_pago = get_text_by_xpath(xp='//label[text()="Forma de pago:"]/following-sibling::label')
# Plazo para entrega de garantía
plazo_garant = get_text_by_xpath(xp='//label[text()="Plazo en días para entregar la garantía:"]/following-sibling::label')
# Número de meses para la garantía
meses_garant = get_text_by_xpath(xp='//label[text()="Número de meses que debe cumplir la garantía:"]/following-sibling::label')
#Garantía de cumplimiento
garant_cump = get_text_by_xpath(xp='//label[text()="Garantía de cumplimiento:"]/following-sibling::label')
# Carácter
caracter = get_text_by_xpath(xp='//label[text()="Carácter:"]/following-sibling::label')
# Caso fortuito o fuerza mayor
caso_fort  = get_text_by_xpath(xp='//label[text()="Caso fortuito o fuerza mayor:"]/following-sibling::label')
# Tipo de contrato abierto
tipo_cont_abierto = get_text_by_xpath(xp='//label[text()="Tipo de contrato abierto:"]/following-sibling::label')
#oportunnity url
uri = driver.current_url

In [ ]:
today = date.today().strftime("%d/%m/%Y")

In [ ]:
import pandas as pd

In [ ]:
def load_csv(file_name:str) -> pd.DataFrame:
    #lee la tabla de la pagina
    return pd.read_csv(file_name, encoding='utf-8')

In [ ]:
concluidos = load_csv("concluidos.csv")

In [ ]:
concluidos

In [ ]:
from datetime import datetime

In [ ]:
datetime.strptime(fecha_pub, "%d/%m/%Y %H:%M").strftime("%d/%m/%Y %H:%M")

In [ ]:
new_row = {'cod_exp':cod_exp, 'num_proc':num_proc, 'dependencia':dependencia, 'unidad_comp':unidad_comp, 
        'correo':correo, 'nombre_procedimiento':nombre_procedimiento, 'tipo_proc':tipo_proc, 
        'entidad_fed':entidad_fed, 'anio_ej':anio_ej, 'proc_exc':proc_exc, 'fecha_pub':fecha_pub, 
        'claves_list':claves_list, 'part_testigo':part_testigo, 'abastecimiento_sim':abastecimiento_sim, 
        'plurianual':plurianual, 'tipo_cont':tipo_cont, 'anticipo':anticipo, 'forma_pago':forma_pago, 
        'plazo_garant':plazo_garant, 'meses_garant':meses_garant, 'caracter':caracter, 'caso_fort ':caso_fort , 
        'tipo_cont_abierto':tipo_cont_abierto, 'uri':uri, 'scrapped_day':today}

In [ ]:
concluidos = pd.concat([concluidos, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
concluidos.to_csv("concluidos.csv", index=False, encoding='utf-8', header=True)

In [ ]:
anexos = load_csv("anexos.csv")

In [ ]:
new_row = {'cod_exp':cod_exp, 'num_proc':num_proc, 'anexo':filename, 'scrapped_day':today}

In [ ]:
anexos = pd.concat([anexos, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
anexos

In [ ]:
driver.close()